In [3]:
# Install NeuralCoref
!pip install https://github.com/huggingface/neuralcoref-models/releases/download/en_coref_md-3.0.0/en_coref_md-3.0.0.tar.gz

  Running setup.py bdist_wheel for en-coref-md: started
  Running setup.py bdist_wheel for en-coref-md: finished with status 'done'
  Stored in directory: C:\Users\JoshB\AppData\Local\pip\Cache\wheels\aa\a3\8f\9df13c027b75169bcca62682563e9823bb213c72a2cc3efed8
Successfully built en-coref-md


In [17]:
# Need to downgrade SpaCy
!pip install -U spacy==2.0.11

  Running setup.py bdist_wheel for spacy: started
  Running setup.py bdist_wheel for spacy: still running...
  Running setup.py bdist_wheel for spacy: still running...
  Running setup.py bdist_wheel for spacy: finished with status 'done'
  Stored in directory: C:\Users\JoshB\AppData\Local\pip\Cache\wheels\fb\00\28\75c85d5135e7d9a100639137d1847d41e914ed16c962d467e4
  Running setup.py bdist_wheel for pathlib: started
  Running setup.py bdist_wheel for pathlib: finished with status 'done'
  Stored in directory: C:\Users\JoshB\AppData\Local\pip\Cache\wheels\f9\b2\4a\68efdfe5093638a9918bd1bb734af625526e849487200aa171
  Running setup.py bdist_wheel for regex: started
  Running setup.py bdist_wheel for regex: finished with status 'done'
  Stored in directory: C:\Users\JoshB\AppData\Local\pip\Cache\wheels\75\07\38\3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built spacy pathlib regex
  Found existing installation: regex 2018.1.10
    Uninstalling regex-2018.1.10:
      Succe

In [1]:
# Import medium English model
import spacy

nlp = spacy.load('en_coref_md')

In [22]:
# Read in data file
import pandas as pd
data = pd.read_csv('data.tsv', sep='\t')
data.set_index(data.columns[0], inplace=True)

data.head()

,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL
ID,,,,,,,,
development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...
development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie
development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band)
development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera


In [23]:
answers = pd.read_csv('submission.csv')
answers.set_index(answers.columns[0], inplace=True)

# Drop rows where correct answer is neither A nor B
answers = answers[answers['NEITHER'] == 0]

answers = answers[['A', 'B']]

answers.columns = ['answer_A', 'answer_B']
answers.head()

,answer_A,answer_B
ID,,
development-1,1,0
development-2,1,0
development-3,0,1
development-4,0,1
development-5,0,1


In [24]:
data = data.merge(answers, on='ID')
data.head()

,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL,answer_A,answer_B
ID,,,,,,,,,,
development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...,1,0
development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie,1,0
development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...,0,1
development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band),0,1
development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera,0,1


In [25]:
data = data[['answer_A', 'answer_B', 'A-offset', 'B-offset']]
data.head()

,answer_A,answer_B,A-offset,B-offset
ID,,,,
development-1,1,0,191,207
development-2,1,0,228,251
development-3,0,1,173,246
development-4,0,1,174,336
development-5,0,1,219,294


In [26]:
data['offset'] = data['answer_A'] * data['A-offset'] + data['answer_B'] * data['B-offset']
data.head()

,answer_A,answer_B,A-offset,B-offset,offset
ID,,,,,
development-1,1,0,191,207,191
development-2,1,0,228,251,228
development-3,0,1,173,246,246
development-4,0,1,174,336,336
development-5,0,1,219,294,294


In [50]:
offsets = data[['offset']]
offsets.head()

,offset
ID,
development-1,191
development-2,228
development-3,246
development-4,336
development-5,294


In [51]:
coreNLP = pd.read_csv('resolved_words_with_offset_corenlp.csv')
coreNLP.set_index(coreNLP.columns[1], inplace=True)
coreNLP

,Unnamed: 0,OFFSET
ID,,
development-1,0,170
development-2,1,228
development-3,2,164
development-5,3,219
development-6,4,0
development-7,5,152
development-8,6,173
development-9,7,128
development-10,8,196


In [52]:
offsets['CoreNLP_offset'] = coreNLP['OFFSET']
#offsets.dropna(inplace=True)
#offsets.astype({'offset': int, 'CoreNLP_offset': int})
offsets.dropna(inplace=True)
offsets.head()


C:\Users\JoshB\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\JoshB\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,offset,CoreNLP_offset
ID,,
development-1,191,170.0
development-2,228,228.0
development-3,246,164.0
development-5,294,219.0
development-6,236,0.0


In [53]:
offsets['coreNLP_match'] = offsets['offset'] == offsets['CoreNLP_offset']
offsets.head()

C:\Users\JoshB\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,offset,CoreNLP_offset,coreNLP_match
ID,,,
development-1,191,170.0,False
development-2,228,228.0,True
development-3,246,164.0,False
development-5,294,219.0,False
development-6,236,0.0,False


In [54]:
offsets['coreNLP_match'].value_counts()

False    1423
True      322
Name: coreNLP_match, dtype: int64

In [55]:
1423+322

1745

In [63]:
# Rebuild answer key offsets to compare with NeuralCoref
offsets = data[['offset']]
offsets.head()

,offset
ID,
development-1,191
development-2,228
development-3,246
development-4,336
development-5,294


In [64]:
neuralCoref = pd.read_csv('spacy_resolved.csv')
neuralCoref.set_index(neuralCoref.columns[1], inplace=True)
neuralCoref.head()

,Unnamed: 0,mention_offset
ID,,
development-1,0,170
development-2,1,0
development-3,2,98
development-4,3,281
development-5,4,0


In [65]:
import numpy as np
neuralCoref.replace('X', np.nan, inplace=True)
neuralCoref.dropna(inplace=True)

In [66]:
offsets['neural_offset'] = neuralCoref['mention_offset']
#offsets.dropna(inplace=True)
#offsets.astype({'offset': int, 'CoreNLP_offset': int})
offsets.dropna(inplace=True)
offsets.astype(int)
offsets.head()


C:\Users\JoshB\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\JoshB\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,offset,neural_offset
ID,,
development-1,191,170
development-2,228,0
development-3,246,98
development-4,336,281
development-5,294,0


In [80]:
offsets['neural_offset'] = offsets['neural_offset'].astype('int')
offsets['neural_match'] = offsets['offset'] == offsets['neural_offset']
offsets.head()

C:\Users\JoshB\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\JoshB\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,offset,neural_offset,neural_match
ID,,,
development-1,191,170,False
development-2,228,0,False
development-3,246,98,False
development-4,336,281,False
development-5,294,0,False


In [81]:
offsets['neural_match'].value_counts()

False    1361
True      266
Name: neural_match, dtype: int64

In [84]:
offsets.head()

,offset,neural_offset,neural_match
ID,,,
development-1,191,170,False
development-2,228,0,False
development-3,246,98,False
development-4,336,281,False
development-5,294,0,False


In [87]:
c =coreNLP
c['corenlp_offset'] =coreNLP['OFFSET']

In [90]:
n =neuralCoref
n['neuralcoref_offset'] = neuralCoref['mention_offset']

In [93]:
f = pd.concat([c,n], axis=1, join='inner')
f

In [99]:
f[['mention_offset', 'neuralcoref_offset']] = f[['mention_offset', 'neuralcoref_offset']].apply(lambda x: x.astype('int64'))

In [103]:
f['result'] = f['corenlp_offset']==f['neuralcoref_offset']
f['result'].value_counts()

False    1100
True      664
Name: result, dtype: int64

In [104]:
f.shape

(1764, 7)